In [1]:
import pandas as pd
import os
from tqdm import tqdm
import time
import traceback

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
random_surveys_data_directory = "/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/input_data/Daytime Surveys 9.11.23 FINAL.xlsx"
morning_surveys_data_directory = "/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/input_data/Morning Reports 9.11.23 FINAL.xlsx"
participant_id_mapping_directory = "/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/input_data/MetricWire participant ID key_FINAL.xlsx"
column_info_mapping = "/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/input_data/column_mapping.csv"
op_directory = '/Users/sauravyadav/Documents/Repos/Datasets/BAR_EMA/output_data'

In [3]:
random_survey_df = pd.read_excel(random_surveys_data_directory, engine='openpyxl')
morning_survey_df = pd.read_excel(morning_surveys_data_directory, engine='openpyxl')
pid_mapping_df = pd.read_excel(participant_id_mapping_directory, engine='openpyxl')

In [4]:
display(random_survey_df.head(1))
display(morning_survey_df.head(1))
display(pid_mapping_df.head(1))

,Response Id,Response Type,User Id,Study Id,Survey Id,Source Name,Source Type,Survey Started Date,Survey Started Time,Survey Submitted Date,Survey Submitted Time,Time Zone,Trigger Id,Trigger Date,Trigger Time,Trigger Type,Trigger Name,Trigger Index,Device OS,Device OS Version,App Version,Submission Location,Please rate how you are feeling right now:Energized,Excited,Sedated,Slow thoughts,Sluggish,Up,Please rate how you are feeling right now:Cheerful,Annoyed,Upbeat,Sad,Irritated,Happy,Bored,Content,Where are you currently?,Please specify where you are currently.,Are you with others?,How many people are you interacting with?,Who are these people you are interacting with?,Please specify who you are with currently.,How much are you enjoying this interaction?,How are you acting during this interaction?,How are you acting during this interaction? .1,How is the other person acting during this interaction?,How is the other person acting during this interaction? .1,"Are you interacting with someone virtually, on your computer, phone, or the like?",I like being alone right now.,Being alone feels good to me.,I wish I were with others right now.,I feel lonely.,I feel rejected.,I feel included.,I feel accepted.,How many people are you interacting with?.1,Who are these people you are interacting with?.1,Please specify who you are with currently..1,How much are you enjoying this interaction?.1,How are you acting during this interaction?,How are you acting during this interaction?.1,How is the other person acting during this interaction?,How is the other person acting during this interaction? .1,I like that I’m not interacting with anyone right now.,I wish I were interacting with someone right now.,I feel lonely..1,I feel rejected..1,I feel included..1,I feel accepted.,Have you experienced the following things since your last survey?I said things without thinking.,I spent more money than I meant to.,I have felt impatient.,I made a “spur of the moment” decision.,Are you drinking alcohol right now?,"If this is your first drink of the day and you have not already done so, please make sure to initiate the “After You 1st Drink” survey that’s available on the home screen of the app.\nThank you!"
0,5c784be4b05c7d1f4f742a08,Submission,58431e46e75bda1d3007c0c8,5aa83b46284c846391451166,5aa83b46284c84639145132a,Daytime Surveys,Assessment,28/02/2019,15:58:24,28/02/2019,16:00:20,-5:00,5aa83b46284c846391451416,28/02/2019,15:58:08,NaN,NaN,NaN,iOS,12.1.4,4.1.2,"0.00000,0.00000",75.0,87.0,17.0,33.0,37.0,77.0,83.0,26.0,80.0,9.0,21.0,83.0,18.0,96.0,1.0,CONDITION_SKIPPED,2.0,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,1,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,2,3,CONDITION_SKIPPED,86,100,51,100,51,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,63.0,3.0,15.0,81.0,2.0,CONDITION_SKIPPED


,Response Id,Response Type,User Id,Study Id,Survey Id,Source Name,Source Type,Survey Started Date,Survey Started Time,Survey Submitted Date,Survey Submitted Time,Time Zone,Trigger Id,Trigger Date,Trigger Time,Trigger Type,Trigger Name,Trigger Index,Device OS,Device OS Version,App Version,Submission Location,What time did you wake up this morning?,What time did you get out of bed?,What time did you go to bed last night?,How long did it take you to fall asleep?,Last night my sleep quality was…,How many drinks did you have yesterday/last night? Please include drinks that you had after midnight.,How long were you drinking yesterday (from starting your first drink to finishing your last drink)?\n(Example: # hours # minutes).,Next you will see a series of statements. \nPlease indicate if any of the following things happened to you as a result of your drinking yesterday,Felt more relaxed.,Had a hangover.,Did something that embarrassed me.,Was more aggressive.,Felt nauseated or vomited.,Hurt or injured myself by accident.,Was unable to remember what I did.,Was unable to study/do work.,Was rude or obnoxious.,Was more sociable.,Was able to express my feelings more easily.,Had a feeling of guilt or remorse.,Failed to do what was normally expected of me.,Was in a better mood.,Needed a drink this morning to get myself going.,Drank when promised self not to.,Drank much more than I planned to.,Got into an argument or physical fight.,Felt more energetic.,Passed out.,Missed class/work or was late to class/work.,Did impulsive things I regret.,Slept poorly.,Please rate how you are feeling right now:Cheerful,Annoyed,Upbeat,Sad,Irritated,Happy,Bored,Content,Did you smoke marijuana yesterday?,Do you plan to drink alcohol today or tonight?
0,5c793b925775086ef84708f2,Submission,58431e46e75bda1d3007c0c8,5aa83b46284c846391451166,5aa83b46284c846391451285,Morning Reports,Assessment,2019-01-03 00:00:00,09:00:04,2019-01-03 00:00:00,09:02:58,-5:00,5aa83b46284c846391451417,2019-01-03 00:00:00,05:00:00,NaN,NaN,NaN,iOS,12.1.4,4.1.2,"0.00000,0.00000",09:00:00,09:00:00,02:00:00,1.0,3.0,9.0,5 hours,INFORMATION_QUESTION,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,68.0,8.0,74.0,8.0,9.0,85.0,13.0,91.0,2.0,1.0


,SUB ID,MW User ID
0,2001,5a8381835893ca7070656612


### Concatenate Morning and Random reports. Also add Participant ID

In [5]:
res_df = pd.concat([random_survey_df, morning_survey_df]).reset_index(drop=True).merge(pid_mapping_df, left_on='User Id', right_on='MW User ID', how='left').drop(columns='MW User ID').reset_index(drop=True)

### Remove garbage from cols:

In [6]:
column_name_fix = {col: col.replace('\xa0', '').strip() for col in res_df.columns}
res_df.rename(columns=column_name_fix, inplace=True)

### Renaming Duplicate Columns

In [7]:
new_cols = []
seen_cols = []
i=1
for col in  res_df.columns:
    if col in seen_cols:
        new_cols.append(f'{col}_{i}')
        seen_cols.append(f'{col}_{i}')
        i+=1
    else:
        new_cols.append(col)
        seen_cols.append(col)
res_df.columns = new_cols

### Creating a mapping from current to desired columns

In [8]:
# col_mapping_df = pd.read_csv(column_info_mapping)
# value_counts = col_mapping_df['Current variable name'].value_counts().reset_index()
# value_counts = value_counts[value_counts['count'] > 1]
# display(value_counts)

In [9]:
# column_mapping = dict(zip(col_mapping_df["Desired variable names"], col_mapping_df["Current variable name"]))
# print(column_mapping)

In [10]:
inverse_column_mapping = {'SubID': 'SUB ID',
 # 'Brst': 'Burst',
 # 'Wknd': 'Weekend',
 # 'Day': 'Day Within Weekend',
 'RespID': 'Response Id',
 'RespType': 'Response Type',
 'UserID': 'User Id',
 'SurvName': 'Source Name',
 'SurvType': 'Source Type',
 'InitDate': 'Survey Started Date',
 'InitTime': 'Survey Started Time',
 'SubDate': 'Survey Submitted Date',
 'SubTime': 'Survey Submitted Time',
 'TrigDate': 'Trigger Date',
 'TrigTime': 'Trigger Time',
 'MR_wake': 'What time did you wake up this morning?',
 'MR_rise': 'What time did you get out of bed?',
 'MR_bed': 'What time did you go to bed last night?',
 'MR_FlSlp': 'How long did it take you to fall asleep?',
 'MR_SlpQlt': 'Last night my sleep quality was…',
 'MR_DrnkNum': 'How many drinks did you have yesterday/last night? Please include drinks that you had after midnight.',
 'MR_DrnkDur': 'How long were you drinking yesterday (from starting your first drink to finishing your last drink)?\n(Example: # hours # minutes).',
 'MR_rlx': 'Felt more relaxed.',
 'MR_hngvr': 'Had a hangover.',
 'MR_Embrssd': 'Did something that embarrassed me.',
 'MR_agrsv': 'Was more aggressive.',
 'MR_vom': 'Felt nauseated or vomited.',
 'MR_hrt': 'Hurt or injured myself by accident.',
 'MR_rmbr': 'Was unable to remember what I did.',
 'MR_work': 'Was unable to study/do work.',
 'MR_rude': 'Was rude or obnoxious.',
 'MR_soc': 'Was more sociable.',
 'MR_exprs': 'Was able to express my feelings more easily.',
 'MR_guilt': 'Had a feeling of guilt or remorse.',
 'MR_expctd': 'Failed to do what was normally expected of me.',
 'MR_mood': 'Was in a better mood.',
 'MR_mrng': 'Needed a drink this morning to get myself going.',
 'MR_prmsd': 'Drank when promised self not to.',
 'MR_more': 'Drank much more than I planned to.',
 'MR_fght': 'Got into an argument or physical fight.',
 'MR_enrgtc': 'Felt more energetic.',
 'MR_PsdOut': 'Passed out.',
 'MR_MsdClss': 'Missed class/work or was late to class/work.',
 'MR_Impls': 'Did impulsive things I regret.',
 'MR_sleep': 'Slept poorly.',
 'MR_weed': 'Did you smoke marijuana yesterday?',
 'MR_plan': 'Do you plan to drink alcohol today or tonight?',
 'chr': 'Pleaserate how you are feeling right now:Cheerful',
 'anoy': 'Annoyed',
 'upbt': 'Upbeat',
 'sad': 'Sad',
 'irit': 'Irritated',
 'hap': 'Happy',
 'bor': 'Bored',
 'cnt': 'Content',
 'RP_enrgz': 'Pleaserate how you are feeling right now:Energized',
 'RP_excit': 'Excited',
 'RP_sedat': 'Sedated',
 'RP_slotht': 'Slow thoughts',
 'RP_slug': 'Sluggish',
 'RP_up': 'Up',
 'RP_crntloc': 'Where are you currently?',
 'RP_othrloc': 'Please specify where you are currently.',
 'RP_wthothr': 'Are you with others?',
 'RP_NumOth': 'How many people are you interacting with?',
 'RP_OthTyp': 'Who are these people you are interacting with?',
 'RP_OthWho': 'Please specify who you are with currently.',
 'RP_EnjInt': 'How much are you enjoying this interaction?',
 'RP_ActCld': 'How are you acting during this interaction?',
 'RP_ActAcc': 'How are you acting during this interaction?.1',
 'RP_OthCld': 'How is the other person acting during this interaction?',
 'RP_OthAcc': 'How is the other person acting during this interaction?.1',
 'RP_Vrtl': 'Are you interacting with someone virtually, on your computer, phone, or the like?',
 'RP_AlnLke': 'I like being alone right now.',
 'RP_AlnGd': 'Being alone feels good to me.',
 'RP_WshOth': 'I wish I were with others right now.',
 'RP_lnly': 'I feel lonely.',
 'RP_rjct': 'I feel rejected.',
 'RP_incl': 'I feel included.',
 'RP_acpt': 'I feel accepted.',
 'RP_vrtint': 'How many people are you interacting with?.1',
 'RP_vrtoth': 'Who are these people you are interacting with?.1',
 'RP_vrtwho': 'Please specify who you are with currently..1',
 'RP_vrtenj': 'How much are you enjoying this interaction?.1',
 'RP_vrtcld': 'How are you acting during this interaction?_1',
 'RP_VrtAcc': 'How are you acting during this interaction?.1_2',
 'RP_VrtOthCld': 'How is the other person acting during this interaction?_3',
 'RP_VrtOthAcc': 'How is the other person acting during this interaction? .1',
 'RP_NotInt': 'I like that I’m not interacting with anyone right now.',
 'RP_WshInt': 'I wish I were interacting with someone right now.',
 'RP_lonely': 'I feel lonely..1',
 'RP_reject': 'I feel rejected..1',
 'RP_included': 'I feel included..1',
 'RP_accepted': 'I feel accepted._4',
 'RP_said': 'Have you experienced the following things since your last survey?I said things without thinking.',
 'RP_spent': 'I spent more money than I meant to.',
 'RP_impt': 'I have felt impatient.',
 'RP_spur': 'I made a “spur of the moment” decision.',
 'RP_Drnk': 'Are you drinking alcohol right now?'}

column_mapping = { v: k for k,v in inverse_column_mapping.items()}

### Renaming & Reordering Columns to Desired Column Names

In [11]:
res_df = res_df.rename(columns = column_mapping)
res_df = res_df[list(column_mapping.values())]

In [12]:
res_df.sample()

,SubID,RespID,RespType,UserID,SurvName,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,MR_wake,MR_rise,MR_bed,MR_FlSlp,MR_SlpQlt,MR_DrnkNum,MR_DrnkDur,MR_rlx,MR_hngvr,MR_Embrssd,MR_agrsv,MR_vom,MR_hrt,MR_rmbr,MR_work,MR_rude,MR_soc,MR_exprs,MR_guilt,MR_expctd,MR_mood,MR_mrng,MR_prmsd,MR_more,MR_fght,MR_enrgtc,MR_PsdOut,MR_MsdClss,MR_Impls,MR_sleep,MR_weed,MR_plan,chr,anoy,upbt,sad,irit,hap,bor,cnt,RP_enrgz,RP_excit,RP_sedat,RP_slotht,RP_slug,RP_up,RP_crntloc,RP_othrloc,RP_wthothr,RP_NumOth,RP_OthTyp,RP_OthWho,RP_EnjInt,RP_ActCld,RP_ActAcc,RP_OthCld,RP_OthAcc,RP_Vrtl,RP_AlnLke,RP_AlnGd,RP_WshOth,RP_lnly,RP_rjct,RP_incl,RP_acpt,RP_vrtint,RP_vrtoth,RP_vrtwho,RP_vrtenj,RP_vrtcld,RP_VrtAcc,RP_VrtOthCld,RP_VrtOthAcc,RP_NotInt,RP_WshInt,RP_lonely,RP_reject,RP_included,RP_accepted,RP_said,RP_spent,RP_impt,RP_spur,RP_Drnk
48499,2344,40fff2b9-7721-4dc4-9341-f81bebb6ecf4,Missed,5cbe293607a41c22d5b71560,Morning Reports,NaN,23/06/2022,05:00:00,23/06/2022,13:00:00,23/06/2022,05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
res_df.to_csv(f'{op_directory}\prelim_data_0.csv',index=False)